In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics

from sklearn.linear_model import ElasticNet,ElasticNetCV


from warnings import simplefilter

import pickle
from sklearn.externals import joblib

rand_st=4
#simplefilter("ignore") 

In [2]:
#output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx')
output=pd.read_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results.xlsx')
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",186.539067,234.559134,0.486111,0.367,Обучение на полном набооре -> прогноз рейтинго...
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",189.358443,237.420124,0.483303,0.367,Обучение на полном набооре -> прогноз рейтинго...
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",175.593949,221.353796,0.453718,0.164,Обучение только на победах белых -> прогноз ре...
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",176.326439,221.767050,0.461520,0.269,Обучение только на победах черных -> прогноз р...
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",177.431734,224.595199,0.464714,0.151,Обучение только на ничьих -> прогноз ретйтинга...
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",182.048649,228.384992,0.451721,0.142,Обучение только на ничьих -> прогноз ретйтинга...
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",192.062805,241.580990,0.467338,0.151,Обучение только на проигрышах белых -> прогноз...
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",193.242293,242.404488,0.461618,0.165,Обучение только на проигрышах черных -> прогно...
8,"LinearRegression(copy_X=True, fit_intercept=Tr...",171.079101,215.522294,0.461243,0.370,Обучение на среднем ЭЛО
9,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.700896,192.272077,0.574320,0.362,Обучение на разнице в рейтинге


In [3]:
#elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,-57,4765
1,1915,1999,1957,-84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [4]:
#features=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv')
features=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\features.csv')

print(features.info())
print(features.shape)
features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24255 entries, 0 to 24254
Columns: 233 entries, Result to InsufficientMaterial
dtypes: float64(111), int64(122)
memory usage: 43.1 MB
None
(24255, 233)


,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,EndB,EndR,EndK,WhiteСhecks,WhiteQueenCastle,BlackQueenCastle,Promotion,EnPassant,CanClaimDraw,InsufficientMaterial
count,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,...,24255.000000,24255.00000,24255.0,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000
mean,0.544877,42.252690,30.881103,11.907442,111.901711,83.967223,-794.591878,988.643950,68.913894,314.678490,...,0.631293,1.03892,1.0,2.015584,1.457308,0.910740,0.064069,0.061637,0.035415,0.000783
std,0.417736,16.235593,233.709202,445.214702,2310.851869,32.489872,2126.099646,2275.840558,64.170646,475.478471,...,0.676858,0.74847,0.0,2.874868,4.179615,3.663225,0.291749,0.245589,0.184831,0.027978
min,0.000000,10.000000,-2162.262500,-11447.000000,-12261.000000,19.000000,-12348.000000,-51.000000,0.000000,7.795716,...,0.000000,0.00000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,31.000000,-60.758665,0.000000,-362.000000,61.000000,-482.000000,67.000000,30.000000,61.336328,...,0.000000,0.00000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.500000,40.000000,24.563636,0.000000,0.000000,79.000000,-75.000000,156.000000,49.000000,145.621548,...,1.000000,1.00000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,51.000000,123.987715,26.000000,546.500000,102.000000,-18.000000,700.000000,84.000000,291.317992,...,1.000000,2.00000,1.0,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,165.000000,2073.289256,11259.000000,12337.000000,329.000000,26.000000,12352.000000,784.000000,3962.575036,...,2.000000,2.00000,1.0,71.000000,35.000000,36.000000,4.000000,2.000000,1.000000,1.000000


In [5]:
target_white=elos['WhiteElo']
target_black=elos['BlackElo']
target_mean=elos['MeanElos']
target_diff=elos['DiffElos']
target_sum=elos['SumElos']

In [6]:
X_train_sum, X_test_sum, Y_train_sum, Y_test_sum =ms.train_test_split(features, target_sum, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_sum), len(Y_train_sum)))
print(X_train_sum.shape)
print(Y_train_sum.shape)
print(X_test_sum.shape)
print(Y_test_sum.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 233)
(16978,)
(7277, 233)
(7277,)


In [7]:
X_train_diff, X_test_diff, Y_train_diff, Y_test_diff =ms.train_test_split(features, target_diff, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_diff), len(Y_train_diff)))
print(X_train_diff.shape)
print(Y_train_diff.shape)
print(X_test_diff.shape)
print(Y_test_diff.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 233)
(16978,)
(7277, 233)
(7277,)


#### Для теста

In [14]:
target_sum1=target_sum[:5000]
target_diff1=target_diff[:5000]
features1=features[:5000]

X_train1, X_test1, Y_train1, Y_test1 =ms.train_test_split(features1, target_sum1, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train1), len(Y_train1)))
print(X_train1.shape)
print(Y_train1.shape)
print(X_test1.shape)
print(Y_test1.shape)

Train data size: 3500 instances 
Test data size: 3500 instances
(3500, 233)
(3500,)
(1500, 233)
(1500,)


In [9]:
X_train2, X_test2, Y_train2, Y_test2 =ms.train_test_split(features1, target_diff1, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train2), len(Y_train2)))
print(X_train2.shape)
print(Y_train2.shape)
print(X_test2.shape)
print(Y_test2.shape)

Train data size: 700 instances 
Test data size: 700 instances
(700, 233)
(700,)
(300, 233)
(300,)


In [10]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

In [11]:
def GridTune_sum(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))

    
    grid_best_model = grid.fit(X_train1,Y_train1)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

#### ElasticNet

In [15]:
elastic_sum_params =[{'l1_ratio':[0,0.25,0.5,0.75,1],'alpha':[1,0.5],'fit_intercept':[False]},
                     {'l1_ratio':[0,0.25,0.5,0.75,1],'alpha':[1,0.5],'fit_intercept':[True],'normalize':[True,False]}]

start_elastic_sum = time.time()

elastic_sum=ElasticNet(random_state=rand_st)
elastic_sum_model=GridTune_sum(elastic_sum_params,elastic_sum,5)
elastic_sum_predictions=elastic_sum_model.predict(X_test1)

scores_elastic_sum=predictions_report(Y_test1,elastic_sum_predictions)
full_time_elastic_sum=round(time.time() - start_elastic_sum,3)
print(full_time_elastic_sum, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'alpha': 1, 'fit_intercept': False, 'l1_ratio': 0}

Лучший алгоритм:
ElasticNet(alpha=1, copy_X=True, fit_intercept=False, l1_ratio=0,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, warm_start=False)
MAE = 475.235
RMSE = 597.492
Correlation coefficient = 0.175
27.012 c


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [16]:
start_elastic_sum = time.time()

elastic_sum=ElasticNet(alpha=1, copy_X=True, fit_intercept=False, l1_ratio=0,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, warm_start=False).fit(X_train_sum,Y_train_sum)
elastic_sum_predictions=elastic_sum.predict(X_test_sum)

scores_elastic_sum=predictions_report(Y_test_sum,elastic_sum_predictions)
full_time_elastic_sum=round(time.time() - start_elastic_sum,3)
print(full_time_elastic_sum, 'c')

MAE = 473.818
RMSE = 648.825
Correlation coefficient = 0.154
1.989 c


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [17]:
output = output.append(pd.Series([str(elastic_sum), scores_elastic_sum[0],scores_elastic_sum[1], 
                                  scores_elastic_sum[2][0],full_time_elastic_sum,'ElasticNet по сумме рейтингов'],
                                 index=output.columns ),
                      ignore_index=True)

output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",186.539067,234.559134,0.486111,0.367,Обучение на полном набооре -> прогноз рейтинго...
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",189.358443,237.420124,0.483303,0.367,Обучение на полном набооре -> прогноз рейтинго...
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",175.593949,221.353796,0.453718,0.164,Обучение только на победах белых -> прогноз ре...
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",176.326439,221.767050,0.461520,0.269,Обучение только на победах черных -> прогноз р...
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",177.431734,224.595199,0.464714,0.151,Обучение только на ничьих -> прогноз ретйтинга...
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",182.048649,228.384992,0.451721,0.142,Обучение только на ничьих -> прогноз ретйтинга...
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",192.062805,241.580990,0.467338,0.151,Обучение только на проигрышах белых -> прогноз...
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",193.242293,242.404488,0.461618,0.165,Обучение только на проигрышах черных -> прогно...
8,"LinearRegression(copy_X=True, fit_intercept=Tr...",171.079101,215.522294,0.461243,0.370,Обучение на среднем ЭЛО
9,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.700896,192.272077,0.574320,0.362,Обучение на разнице в рейтинге


In [18]:
#output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results.xlsx',index=False)

In [19]:
with open('elastic_sum.pkl', 'wb') as f:
    pickle.dump(elastic_sum,f) 
                         
'''rf_sum = pickle.loads(model_rf_sum)
rf_sum.predict(X[0])'''

joblib.dump(elastic_sum, 'elastic_sum1.pkl')
#model_sum= joblib.load('filename.pkl') 

['elastic_sum1.pkl']

In [20]:
def GridTune_diff(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))
    
    grid_best_model = grid.fit(X_train2,Y_train2)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [21]:
elastic_diff_params =[{'l1_ratio':[0,0.25,0.5,0.75,1],'alpha':[1,0.5],'fit_intercept':[False]},
                     {'l1_ratio':[0,0.25,0.5,0.75,1],'alpha':[1,0.5],'fit_intercept':[True],'normalize':[True,False]}]

start_elastic_diff = time.time()

elastic_diff=ElasticNet(random_state=rand_st)
elastic_diff_model=GridTune_diff(elastic_diff_params,elastic_diff,5)
elastic_diff_predictions=elastic_diff_model.predict(X_test2)

scores_elastic_diff=predictions_report(Y_test2,elastic_diff_predictions)
full_time_elastic_diff=round(time.time() - start_elastic_diff,3)
print(full_time_elastic_diff, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'alpha': 0.5, 'fit_intercept': True, 'l1_ratio': 1, 'normalize': False}

Лучший алгоритм:
ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=1,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, warm_start=False)
MAE = 186.546
RMSE = 279.186
Correlation coefficient = 0.353
10.866 c


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [22]:
start_elastic_diff = time.time()

elastic_diff=ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=1,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, warm_start=False).fit(X_train_diff,Y_train_diff)
elastic_diff_predictions=elastic_diff.predict(X_test_diff)

scores_elastic_diff=predictions_report(Y_test_diff,elastic_diff_predictions)
full_time_elastic_diff=round(time.time() - start_elastic_diff,3)
print(full_time_elastic_diff, 'c')

MAE = 152.512
RMSE = 195.757
Correlation coefficient = 0.556
2.809 c


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [23]:
output = output.append(pd.Series([str(elastic_diff), scores_elastic_diff[0],scores_elastic_diff[1], 
                                  scores_elastic_diff[2][0],full_time_elastic_diff,'ElasticNet на разнице в рейтинге'],
                                 index=output.columns ),
                      ignore_index=True)

output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",186.539067,234.559134,0.486111,0.367,Обучение на полном набооре -> прогноз рейтинго...
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",189.358443,237.420124,0.483303,0.367,Обучение на полном набооре -> прогноз рейтинго...
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",175.593949,221.353796,0.453718,0.164,Обучение только на победах белых -> прогноз ре...
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",176.326439,221.767050,0.461520,0.269,Обучение только на победах черных -> прогноз р...
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",177.431734,224.595199,0.464714,0.151,Обучение только на ничьих -> прогноз ретйтинга...
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",182.048649,228.384992,0.451721,0.142,Обучение только на ничьих -> прогноз ретйтинга...
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",192.062805,241.580990,0.467338,0.151,Обучение только на проигрышах белых -> прогноз...
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",193.242293,242.404488,0.461618,0.165,Обучение только на проигрышах черных -> прогно...
8,"LinearRegression(copy_X=True, fit_intercept=Tr...",171.079101,215.522294,0.461243,0.370,Обучение на среднем ЭЛО
9,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.700896,192.272077,0.574320,0.362,Обучение на разнице в рейтинге


In [24]:
#output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results.xlsx',index=False)

In [25]:
with open('elastic_diff.pkl', 'wb') as f:
    pickle.dump(elastic_diff,f) 
                         
'''rf_sum = pickle.loads(model_rf_sum)
rf_sum.predict(X[0])'''

joblib.dump(elastic_diff, 'elastic_diff1.pkl')
#model_sum= joblib.load('filename.pkl') 

['elastic_diff1.pkl']

#### ElasticNetCV

In [27]:
cvelastic_sum_params =[{'l1_ratio':[0.25,0.5,0.75,1],'n_alphas':[1,2,3],'fit_intercept':[False]},
                     {'l1_ratio':[0.25,0.5,0.75,1],'n_alphas':[1,2,3],'fit_intercept':[True],'normalize':[True,False]}]

start_cvelastic_sum = time.time()

cvelastic_sum=ElasticNetCV(random_state=rand_st,n_jobs=-1,cv=10)
cvelastic_sum_model=GridTune_sum(cvelastic_sum_params,cvelastic_sum,5)
cvelastic_sum_predictions=cvelastic_sum_model.predict(X_test1)

scores_cvelastic_sum=predictions_report(Y_test1,cvelastic_sum_predictions)
full_time_cvelastic_sum=round(time.time() - start_cvelastic_sum,3)
print(full_time_cvelastic_sum, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'fit_intercept': False, 'l1_ratio': 0.25, 'n_alphas': 3}

Лучший алгоритм:
ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=False,
       l1_ratio=0.25, max_iter=1000, n_alphas=3, n_jobs=-1,
       normalize=False, positive=False, precompute='auto', random_state=4,
       selection='cyclic', tol=0.0001, verbose=0)
MAE = 4200.268
RMSE = 4328.936
Correlation coefficient = -0.083
34.166 c


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [31]:
start_cvelastic_sum = time.time()

cvelastic_sum=ElasticNetCV(copy_X=True, fit_intercept=False,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=4, selection='cyclic', tol=0.0001, n_jobs=-1,cv=10).fit(X_train_sum,Y_train_sum)
cvelastic_sum_predictions=cvelastic_sum.predict(X_test_sum)

scores_cvelastic_sum=predictions_report(Y_test_sum,cvelastic_sum_predictions)
full_time_cvelastic_sum=round(time.time() - start_cvelastic_sum,3)
print(full_time_cvelastic_sum, 'c')

MAE = 4190.585
RMSE = 4298.383
Correlation coefficient = -0.108
164.861 c


ElasticNetCv похоже вообще не сходится

#### Проверки

In [32]:
white_elos = (elastic_sum_predictions + elastic_diff_predictions)/2
black_elos = (elastic_sum_predictions-elastic_diff_predictions)/2
pred_elos=list(zip(white_elos,black_elos))
pred_elos[:5],elos[:5]

([(2360.6502452395257, 2496.6555505752826),
  (2013.0746465730329, 2157.7274603706023),
  (1930.8351342469796, 1985.504970071759),
  (1825.4411704839565, 1747.752483942391),
  (2011.7316790275925, 1983.123334310639)],
    WhiteElo  BlackElo  MeanElos  DiffElos  SumElos
 0      2354      2411      2382       -57     4765
 1      1915      1999      1957       -84     3914
 2      2446      2191      2318       255     4637
 3      2168      2075      2121        93     4243
 4      2437      2254      2345       183     4691)

In [33]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
        count+=2
    return round(er/count,5)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
    return round(er,5)

In [34]:
find_abs_error(pred_elos,elos),find_mean_error(pred_elos,elos)

(356808.74387, 24.5162)